In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import to_categorical

# Set the path to your dataset
DATA_DIR = 'data'

# Function to load and label the images
def load_and_label_images(data_dir):
    data = []
    labels = []

    for label, sign_folder in enumerate(os.listdir(data_dir)):
        sign_folder_path = os.path.join(data_dir, sign_folder)

        if os.path.isdir(sign_folder_path):
            for img_file in os.listdir(sign_folder_path):
                img_path = os.path.join(sign_folder_path, img_file)

                # Check if the path is a file and ends with a valid image extension
                if os.path.isfile(img_path) and img_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                    # Read the image
                    img = cv2.imread(img_path)

                    # Check if the image is loaded successfully
                    if img is not None:
                        # Resize the image
                        img = cv2.resize(img, (64, 64))  # Adjust the size as needed

                        # Append the image and label
                        data.append(img)
                        labels.append(label)
                    else:
                        print(f"Error: Could not read the image {img_path}")
                else:
                    print(f"Skipping non-image file: {img_path}")

    return np.array(data), np.array(labels)

# Load and label the images
data, labels = load_and_label_images(DATA_DIR)

# Split the data into training and testing sets
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Convert labels to categorical format
label_encoder = LabelEncoder()
labels_train_encoded = label_encoder.fit_transform(labels_train)
labels_test_encoded = label_encoder.transform(labels_test)
num_classes = len(label_encoder.classes_)

# Normalize pixel values to be between 0 and 1
data_train_normalized = data_train / 255.0
data_test_normalized = data_test / 255.0

# Define the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(data_train_normalized, labels_train_encoded, epochs=30, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(data_test_normalized, labels_test_encoded)
print(f'Test Accuracy: {test_accuracy}')

# Save the trained model
model.save('sign_language_cnn_model.h5')



Skipping non-image file: data\0\.ipynb_checkpoints
Skipping non-image file: data\1\.ipynb_checkpoints



Epoch 1/30


8/8 [==============================] - 2s 111ms/step - loss: 2.0496 - accuracy: 0.2305 - val_loss: 1.9526 - val_accuracy: 0.1875
Epoch 2/30
8/8 [==============================] - 0s 38ms/step - loss: 1.6294 - accuracy: 0.4648 - val_loss: 1.3053 - val_accuracy: 0.5781
Epoch 3/30
8/8 [==============================] - 0s 37ms/step - loss: 0.8924 - accuracy: 0.7852 - val_loss: 0.4039 - val_accuracy: 0.9844
Epoch 4/30
8/8 [==============================] - 0s 38ms/step - loss: 0.2878 - accuracy: 0.9297 - val_loss: 0.1075 - val_accuracy: 0.9844
Epoch 5/30
8/8 [==============================] - 0s 38ms/step - loss: 0.1274 - accuracy: 0.9609 - val_loss: 0.0847 - val_accuracy: 0.9844
Epoch 6/30
8/8 [==============================] - 0s 36ms/step - loss: 0.0488 - accuracy: 0.9922 - val_loss: 0.0707 - val_accuracy: 0.9844
Epoch 7/30
8/8 [==============================] - 0s 36ms

C:\Users\prath\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
len(data)

400

In [3]:
len(labels)

400

In [4]:
import cv2
import numpy as np
import pickle

# Load the trained model
with open('sign_language_model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

# Define the region of interest (ROI) dimensions
roi_top = 100
roi_bottom = 300
roi_left = 150
roi_right = 350

# Define a mapping between original class labels and display names
class_mapping = {
    0: 'Goodluck',
    1: 'good job',
    2: 'dislike',
    3: 'you',
    4: 'highfive',
    5: 'namaste',
    6: 'superb',
    7: 'peace'
}

# Open a connection to the camera (0 is usually the default webcam)
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Flip the frame horizontally for a later selfie-view display
    frame = cv2.flip(frame, 1)

    # Define the region of interest
    roi = frame[roi_top:roi_bottom, roi_left:roi_right]

    # Preprocess the image (resize, flatten)
    roi = cv2.resize(roi, (64, 64))
    roi_flattened = roi.flatten()

    # Make a prediction
    prediction = model.predict([roi_flattened])

    # Map the original class label to the display name
    display_name = class_mapping.get(prediction[0], 'Unknown')

    # Display the prediction as text on the frame
    cv2.putText(frame, f"Prediction: {display_name}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Sign Language Prediction', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()


TypeError: 'NoneType' object is not subscriptable

In [ ]:
data